In [192]:
import os
from pyresparser import ResumeParser

In [219]:
data = ResumeParser('Curriculum Vitae -Shehan Krishan.pdf').get_extracted_data()

c:\Users\sheha\anaconda3\envs\ADHD\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.9). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [220]:
data

{'name': 'Shehan Krishan',
 'email': 'shehankrishan6@gmail.com',
 'mobile_number': '0767158801',
 'skills': ['Etl',
  'Sql',
  'Mysql',
  'Css',
  'Python',
  'Technical skills',
  'Javascript',
  'Tableau',
  'Java',
  'Apis',
  'Research',
  'Html',
  'Data analysis',
  'Pandas',
  'International',
  'Analytical',
  'Database',
  'Ai',
  'Aws',
  'Flask',
  'Startup',
  'Api',
  'Time management',
  'Oracle',
  'Github',
  'Automation',
  'Technical',
  'Mobile',
  'Analysis',
  'Information technology',
  'Programming',
  'Machine learning'],
 'college_name': None,
 'degree': ['B.Sc. Special (Hons) – Information'],
 'designation': ['Senior Software Engineer',
  'Junior Ml Engineer',
  'Pro Vice',
  'AI /Ml Engineer'],
 'experience': ['●',
  '●',
  'Selected for top 10 teams in',
  'Datarush 1.0 organized by',
  'DSSC of SLIIT.',
  'Selected for the Top 10',
  'teams of Intellihack 2.0',
  'organised by IEEE of',
  'University of Colombo.',
  'SOFT SKILLS:',
  'Quick learner',
  'Tea

In [221]:
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import io

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as fh:
        # iterate over all pages of PDF document
        for page in PDFPage.get_pages(fh, caching=True, check_extractable=True):
            # creating a resoure manager
            resource_manager = PDFResourceManager()
            
            # create a file handle
            fake_file_handle = io.StringIO()
            
            # creating a text converter object
            converter = TextConverter(
                                resource_manager, 
                                fake_file_handle, 
                                codec='utf-8', 
                                laparams=LAParams()
                        )

            # creating a page interpreter
            page_interpreter = PDFPageInterpreter(
                                resource_manager, 
                                converter
                            )

            # process current page
            page_interpreter.process_page(page)
            
            # extract text
            text = fake_file_handle.getvalue()
            yield text

            # close open handles
            converter.close()
            fake_file_handle.close()



In [222]:
text = ""
for page in extract_text_from_pdf("Curriculum Vitae -Shehan Krishan.pdf"):
    text += ' ' + page


In [223]:
text

' Shehan Krishan\nData Science Graduate\n\nContact:\n\n0767158801\n\nshehankrishan6@gmail.com\n\n34/25 Kelaninadee rd,Mulleriyawa \n\nhttps://www.linkedin.com/in/shehan-\nkrishan/\n\nTECHNICAL SKILLS:\n\nProgramming Languages\n\n●\n●\n●\n\nPython\nDart\nJavascript\n\nProfessional Profile:\n\nA motivated and results-oriented Ml Engineer with one \n\nyear of experience in AI /ML ,Web Development, API \n\nDevelopment,Mobile App Development . Possesses a \n\nstrong a strong Problem Solving,Time Management \n\nand Analytical Thinking.Adept at AI, Machine Learning, \n\nFlutter, Python, and API development.I am a mature \n\nteam  worker and adaptable to all challenging \n\nsituations. I can work well  both in a team environment \n\nas well as using my own initiative.Seeking to leverage \n\nmy skills and experience to contribute effectively to \n\nyour company where my skills can shine. Moreover,I \n\nam looking for an opportunity that offers me to \n\ndevelop  new skills while strengthening t

In [224]:
import re

def extract_mobile_number(text):
    phone = re.findall(re.compile(r'(?:(?:\+?([1-9]|[0-9][0-9]|[0-9][0-9][0-9])\s*(?:[.-]\s*)?)?(?:\(\s*([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|([0-9][1-9]|[0-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?([2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?([0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?'), text)
    
    if phone:
        number = ''.join(phone[0])
        if len(number) > 10:
            return '+' + number
        else:
            return number

In [225]:
extract_mobile_number(text)

'076715880'

In [226]:
def extract_email(email):
    email = re.findall("([^@|\s]+@[^@]+\.[^@|\s]+)", email)
    if email:
        try:
            return email[0].split()[0].strip(';')
        except IndexError:
            return None

In [227]:
extract_email(text)

'shehankrishan6@gmail.com'

In [228]:
import spacy
import pandas as pd

nlp = spacy.load('en_core_web_sm')

def extract_skills(resume_text):
    nlp_text = nlp(resume_text)

    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    
    # reading the csv file
    data = pd.read_csv("skills.csv") 
    
    # extract values
    skills = list(data.columns.values)
    
    skillset = []
    
    # check for one-grams (example: python)
    for token in tokens:
        if token.lower() in skills:
            skillset.append(token)
    
    # check for bi-grams and tri-grams (example: machine learning)
    for chunk in nlp_text.noun_chunks:
        chunk_text = chunk.text.lower().strip()
        if chunk_text in skills:
            skillset.append(chunk_text)
    
    return list(set([i.capitalize() for i in skillset]))



In [229]:
skills = extract_skills(text)
print(skills) 

['Github', 'Technical skills', 'Analysis', 'Automation', 'Startup', 'Ai', 'Database', 'Java', 'Oracle', 'Css', 'Technical', 'Machine learning', 'Flask', 'Programming', 'Research', 'Tableau', 'Sql', 'Javascript', 'Pandas', 'Flutter', 'Apis', 'Mysql', 'Python', 'Html', 'Api', 'Mobile', 'Data analysis', 'Analytical', 'Etl', 'Aws', 'International', 'Time management']


In [208]:
import re
import spacy

nlp = spacy.load('en_core_web_sm')

EDUCATION = ['BSC', 'B.Tech', 'B.E.', 'M.Sc.', 'M.Tech', 'M.E.', 'Ph.D.']
STOPWORDS = ['and', 'in', 'of', 'at']


def extract_education(resume_text):
    nlp_text = nlp(resume_text)

    # Sentence Tokenizer
    nlp_text = [sent.text.strip() for sent in nlp_text.sents]
    # nlp_text = [sent.string.strip() for sent in nlp_text.sents]
    print("nlp_text",nlp_text)
    
    edu = {}
    # Extract education degree
    for index, text in enumerate(nlp_text[:-1]):
        # print(text)
        text = re.sub(r'[?|$|.|!|,]', r'', text)
        # print(text)

        next_text = nlp_text[index + 1]
        # print("next_text",next_text)
        next_text = re.sub(r'[?|$|.|!|,]', r'', next_text)
        # print("next_text",next_text)
        for tex in text.split():
            # print("tex",tex)
            if tex.upper() in EDUCATION and tex not in STOPWORDS:
                print("tex",tex)
                print("next_text",str(next_text))
                cleaned_text = re.sub(r'●\n', '', next_text)
                print(cleaned_text)
                edu[tex] = cleaned_text
    print(edu)

    # Extract year
    education = []
    for key in edu.keys():
        year = re.search(r'(\b(?:20|19)\d{2}\b)', edu[key])
        if year:
            education.append((key, year.group(0)))
        else:
            education.append(key)
    return education,edu

# Example usage:
resume_text = """
B.Sc. Special (Hons) – Information Technology (Specialization – Data Science) at Sri Lanka Institute of Information Technology (SLIIT) Graduate. (2019 - 2023)
"""
education,edu = extract_education(resume_text)
print(education)


nlp_text ['B.Sc.', 'Special (Hons) – Information Technology (Specialization – Data Science) at Sri Lanka Institute of Information Technology (SLIIT) Graduate.', '(2019 - 2023)']
tex BSc
next_text Special (Hons) – Information Technology (Specialization – Data Science) at Sri Lanka Institute of Information Technology (SLIIT) Graduate
Special (Hons) – Information Technology (Specialization – Data Science) at Sri Lanka Institute of Information Technology (SLIIT) Graduate
{'BSc': 'Special (Hons) – Information Technology (Specialization – Data Science) at Sri Lanka Institute of Information Technology (SLIIT) Graduate'}
['BSc']


In [209]:
education,edu = extract_education(text)
print(education)

nlp_text ['Prabhashwara', 'Karunarathne', 'prabhashwarabandara665@gmaill.com', '+94716201971', '52,Viduhalpathana, Bandarawela \n\nlinkedin.com/in/prabhashwara-\nkarunarathne \n     N7365984', 'Achievements', '•', 'Gold Star Award winner in 2008', '•', 'Chess Champion of India and Sri \nLanka in 2011', 'Hobbies', '•', '•', '•', 'Photography', 'Travel', 'Listening to music', 'Skill Highlights', '•', '•', '•', '•', '•', 'Customer-focused and solutions', 'driven.', 'Strong decision maker', 'Complex analytical person', 'Innovative \nExcellent interpersonal and \ncommunication skills', 'Summary', 'I am a resolute, Organized, and Methodical individual.', 'I am seeking a challenge \noffering  growth  and  potential  in  a  progressive  organization.', 'I  am  a  motivated,', 'highly enthusiastic, hardworking individual who consistently strives to enhance \nmy learning and personal development whenever possible.', 'Experience \n\nExecutive-', 'Recovery Services', 'HNB Finance PLC \n(2023 Octob

In [210]:
text

'  Prabhashwara \nKarunarathne \n\nprabhashwarabandara665@gmaill.com \n\n+94716201971 \n\n52,Viduhalpathana, Bandarawela \n\nlinkedin.com/in/prabhashwara-\nkarunarathne \n     N7365984 \n\nAchievements \n\n•  Gold Star Award winner in 2008 \n• \nChess Champion of India and Sri \nLanka in 2011 \n\nHobbies \n\n• \n• \n• \n\nPhotography \nTravel \nListening to music \n\nSkill Highlights \n\n• \n\n• \n\n• \n• \n• \n\nCustomer-focused and solutions \ndriven. \nStrong decision maker \n\nComplex analytical person \nInnovative \nExcellent interpersonal and \ncommunication skills \n\nSummary \nI am a resolute, Organized, and Methodical individual. I am seeking a challenge \noffering  growth  and  potential  in  a  progressive  organization.  I  am  a  motivated, \nhighly enthusiastic, hardworking individual who consistently strives to enhance \nmy learning and personal development whenever possible. \n\nExperience \n\nExecutive- Recovery Services  \nHNB Finance PLC \n(2023 October – Present) \n

In [211]:
new_text = re.sub(r'●\n', ' ', text)

In [212]:
new_text = re.sub(r'\n', ' ', new_text)

In [213]:
new_text = re.sub(r':', ' - ', new_text)

In [214]:
new_text

'  Prabhashwara  Karunarathne   prabhashwarabandara665@gmaill.com   +94716201971   52,Viduhalpathana, Bandarawela   linkedin.com/in/prabhashwara- karunarathne       N7365984   Achievements   •  Gold Star Award winner in 2008  •  Chess Champion of India and Sri  Lanka in 2011   Hobbies   •  •  •   Photography  Travel  Listening to music   Skill Highlights   •   •   •  •  •   Customer-focused and solutions  driven.  Strong decision maker   Complex analytical person  Innovative  Excellent interpersonal and  communication skills   Summary  I am a resolute, Organized, and Methodical individual. I am seeking a challenge  offering  growth  and  potential  in  a  progressive  organization.  I  am  a  motivated,  highly enthusiastic, hardworking individual who consistently strives to enhance  my learning and personal development whenever possible.   Experience   Executive- Recovery Services   HNB Finance PLC  (2023 October – Present)   •  Manage  the  entire  process  of  letter  correspondence

In [215]:
education,edu = extract_education(new_text)
print(education)

nlp_text ['Prabhashwara', 'Karunarathne', 'prabhashwarabandara665@gmaill.com', '+94716201971', '52,Viduhalpathana, Bandarawela   linkedin.com/in/prabhashwara- karunarathne       N7365984', 'Achievements', '•', 'Gold Star Award winner in 2008', '•', 'Chess Champion of India and Sri  Lanka in 2011', 'Hobbies', '•', '•', '•', 'Photography', 'Travel  Listening to music', 'Skill Highlights', '•', '•', '•', '•', '•', 'Customer-focused and solutions', 'driven.', 'Strong decision maker   Complex analytical person', 'Innovative  Excellent interpersonal and  communication skills', 'Summary', 'I am a resolute, Organized, and Methodical individual.', 'I am seeking a challenge  offering  growth  and  potential  in  a  progressive  organization.', 'I  am', 'a  motivated,  highly enthusiastic, hardworking individual who consistently strives to enhance  my learning and personal development whenever possible.', 'Experience', 'Executive-', 'Recovery Services', 'HNB Finance PLC  (2023 October – Present)'

In [216]:
EDUCATION = ['BSC', 'B.Tech', 'B.E.', 'M.Sc.', 'M.Tech', 'M.E.', 'Ph.D.', 'DIPLOMA']
STOPWORDS = ['and', 'in', 'of', 'at']


def extract_education(resume_text):
    nlp_text = nlp(resume_text)

    # Sentence Tokenizer
    nlp_text = [sent.text.strip() for sent in nlp_text.sents]
    # nlp_text = [sent.string.strip() for sent in nlp_text.sents]
    print("nlp_text",nlp_text)
    
    edu = {}
    # Extract education degree
    for index, text in enumerate(nlp_text[:-1]):
        # print(text)
        text = re.sub(r'[?|$|.|!|,]', r'', text)
        # print(text)

        # next_text = nlp_text[index + 1]
        # print("next_text",next_text)
        # next_text = re.sub(r'[?|$|.|!|,]', r'', next_text)
        # print("next_text",next_text)
        for tex in text.split():
            # print("tex",tex)
            if tex.upper() in EDUCATION and tex not in STOPWORDS:
                next_text = nlp_text[index + 1]
                next_text = re.sub(r'[?|$|.|!|,]', r'', next_text)
                cleaned_text = re.sub(r'●\n', '', next_text)
                print(cleaned_text)
                # for x in range(len(nlp_text) - index):
                #     print(x)
                #     next_text = nlp_text[index + x]
                #     print("next_text",next_text)
                #     years = re.findall(r'\b\d{4}\b', next_text)
                #     if years:
                #         edu["years"] = years
                #         break
                edu[tex] = cleaned_text
                # edu["gg"] = cleaned_text
    print(edu)

    # Extract year
    education = []
    # for key in edu.keys():
    #     year = re.search(r'(\b(?:20|19)\d{2}\b)', edu[key])
    #     if year:
    #         education.append((key, year.group(0)))
    #     else:
    #         education.append(key)
    return education,edu

In [167]:
education,edu = extract_education(new_text)
print(education)

nlp_text ['Shehan Krishan Data Science Graduate', 'Contact -   0767158801', 'shehankrishan6@gmail.com', '34/25 Kelaninadee rd,Mulleriyawa   https - //www.linkedin.com/', 'in/shehan-', 'krishan/', 'TECHNICAL SKILLS -   Programming Languages', 'Python Dart Javascript', 'Professional Profile -', 'A motivated and results-oriented Ml Engineer with one   year of experience in AI', '/ML ,Web Development, API   Development,Mobile App Development .', 'Possesses a   strong a strong Problem Solving,Time Management   and Analytical Thinking.', 'Adept at AI, Machine Learning,   Flutter, Python, and API development.', 'I am a mature   team  worker and adaptable to all challenging   situations.', 'I can work well  both in a team environment   as well as using my own initiative.', 'Seeking to leverage   my skills and experience to contribute effectively to   your company where my skills can shine.', 'Moreover,I   am looking for an opportunity that offers me to   develop  new skills while strengthening

In [217]:
text

'  Prabhashwara \nKarunarathne \n\nprabhashwarabandara665@gmaill.com \n\n+94716201971 \n\n52,Viduhalpathana, Bandarawela \n\nlinkedin.com/in/prabhashwara-\nkarunarathne \n     N7365984 \n\nAchievements \n\n•  Gold Star Award winner in 2008 \n• \nChess Champion of India and Sri \nLanka in 2011 \n\nHobbies \n\n• \n• \n• \n\nPhotography \nTravel \nListening to music \n\nSkill Highlights \n\n• \n\n• \n\n• \n• \n• \n\nCustomer-focused and solutions \ndriven. \nStrong decision maker \n\nComplex analytical person \nInnovative \nExcellent interpersonal and \ncommunication skills \n\nSummary \nI am a resolute, Organized, and Methodical individual. I am seeking a challenge \noffering  growth  and  potential  in  a  progressive  organization.  I  am  a  motivated, \nhighly enthusiastic, hardworking individual who consistently strives to enhance \nmy learning and personal development whenever possible. \n\nExperience \n\nExecutive- Recovery Services  \nHNB Finance PLC \n(2023 October – Present) \n

In [172]:
kk= "Special (Hons) – Information   Technology (Specialization – Data Science)   at Sri Lanka Institute of Information   Technology (SLIIT) Graduate"
years = re.findall(r'\b\d{4}\b', kk)
print(years)

[]


In [218]:
education,edu = extract_education(new_text)
print(education)

nlp_text ['Prabhashwara', 'Karunarathne', 'prabhashwarabandara665@gmaill.com', '+94716201971', '52,Viduhalpathana, Bandarawela   linkedin.com/in/prabhashwara- karunarathne       N7365984', 'Achievements', '•', 'Gold Star Award winner in 2008', '•', 'Chess Champion of India and Sri  Lanka in 2011', 'Hobbies', '•', '•', '•', 'Photography', 'Travel  Listening to music', 'Skill Highlights', '•', '•', '•', '•', '•', 'Customer-focused and solutions', 'driven.', 'Strong decision maker   Complex analytical person', 'Innovative  Excellent interpersonal and  communication skills', 'Summary', 'I am a resolute, Organized, and Methodical individual.', 'I am seeking a challenge  offering  growth  and  potential  in  a  progressive  organization.', 'I  am', 'a  motivated,  highly enthusiastic, hardworking individual who consistently strives to enhance  my learning and personal development whenever possible.', 'Experience', 'Executive-', 'Recovery Services', 'HNB Finance PLC  (2023 October – Present)'

In [190]:
edu

{'BSc': 'Special (Hons) – Information   Technology (Specialization – Data Science)   at Sri Lanka Institute of Information   Technology (SLIIT) Graduate',
 'Diploma': '(2018)  GCE (Advance Level) Exam at Sri Subuthi'}

In [108]:
import re

text = "\n\n●\n\n●\n\n●\n\n●\n\nB.Sc. Special (Hons) – Information \n\nTechnology \n\n●\n\n●\n\n●\n\n●\n\n"

cleaned_text = re.sub(r'(\n\n●)+|\n\n', '', text).strip()
print(cleaned_text)

B.Sc. Special (Hons) – Information Technology
